![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor


# import
Импортируем нужные библиотеки

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import os
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# Загружаем специальный удобный инструмент для разделения датасета:


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [4]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')

sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

FileNotFoundError: [Errno 2] File /kaggle/input/sf-dst-restaurant-rating//main_task.csv does not exist: '/kaggle/input/sf-dst-restaurant-rating//main_task.csv'

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1  # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
# в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями
df_test['Rating'] = 0

data = df_test.append(df_train, sort=False).reset_index(
    drop=True)  # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   


## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(
    data['Number of Reviews']).astype('uint8')

In [ ]:
data['Number_of_Reviews_isNAN']

In [ ]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=['City', ], dummy_na=True)

In [ ]:
data.head(5)

In [ ]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [ ]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [ ]:
# Ваша обработка 'Price Range'
def Price_range(price):
    if price == '$':
        rank = 1
    elif price == '$$ - $$$':
        rank = 2
    elif price == '$$$$':
        rank = 3
    else:
        rank = 0
    return rank


data['Price Range'] = data['Price Range'].apply(Price_range)
data['Price Range'].value_counts()

Учитывая количество ресторанов с ценовой категорией 2 и количество нулей, которые сильно превышают количество ресторанов с рангом 1 и 2, не будет большой ошибкой заменить нули модой, то есть присвоить им ранг 2.
Казалось бы, мелочь, но точность модели увеличилась.

In [ ]:
data['Price Range'] = data['Price Range'].apply(lambda x: 2 if x == 0 else x)
data['Price Range'].value_counts()
savePR = data['Price Range']  # сохраню, а то растащат на dummy-переменные

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

### Dummy-переменные для ценовых рангов

In [ ]:
data = pd.get_dummies(data, columns=['Price Range'], dummy_na=False)
data['Price Range'] = savePR  # возвращаем столбец на место
data.head()

Как оказалось, ценовые ранги мало влияют на работу модели. В принципе, их можно даже удалить. Но раз уж написал, да ещё всего 4 столбца - пусть будут.

### Количество кухонь в каждом ресторане

In [ ]:
# Создаём признак показывающий, сколько видов кухонь в каждом ресторане. Если данных нет, то это количество считается за 1
a = data['Cuisine Style'].apply(lambda x: str(x)[2:-2].split("', '"))
Cuisine_assort = a.apply(lambda x: (len(x)))
data['Cuisine_assort'] = Cuisine_assort

### Dummy-переменные для всех видов кухонь
Их 125, немало, но и не сильно много, сделаем dummy-переменные для всех.

Как оказалось в дальнейшем, на MAE эти данные не повлияли практически никак, и ни один из столбцов не вошёл в 20 важных для модели, так что этот кусок кода я исключил из обработки. Посетителям ресторана, похоже, важен ассортимент, а какая-то конкретная кухня их не интересует)

In [ ]:
'''#Создаём список, в котором находятся списки кухонь для каждого ресторана
all_rest = [] # это будет список списков кухонь для каждого ресторана
cuisine_list = set() # а это будет список всех уникальных кухонь
cuisine = data['Cuisine Style'].dropna()
for i in cuisine:
    i = i[2:-2].split("', '")
    all_rest.append(i)
#display(all_rest)

for i in all_rest:
    for j in i:
        cuisine_list.add(j)

CDum = pd.get_dummies(list(cuisine_list))
for i in CDum:
    data[i] = CDum[i]
                   
data.head()'''

### Разница во времени между двумя последними отзывами

In [ ]:
reviews = data['Reviews'].apply(
    lambda x: x[2:-2].split('], [')[1][1:-1].split("', '") if pd.isna(x) == False else [0])

# заполним константой, в расчётах она всё равно превратится в 0.
reviews = reviews.apply(lambda x: x if x[0] != '' else ['01/01/1900'])

deltarev = []

for i in reviews:
    if len(i) > 1:
        i[0] = datetime.strptime(i[0], '%m/%d/%Y')
        i[1] = datetime.strptime(i[1], '%m/%d/%Y')
        d = abs((i[1]-i[0])).days
        deltarev.append(d)
    else:
        deltarev.append(0)

data['ReviewTimeDelta'] = deltarev

data['ReviewTimeDelta'].sample(10)

### Время, прошедшее с последнего отзыва
Так, а что, ежели ещё один признак будет разницей между последним отзывом и вот прям щас?
Немножко поменяем предыдущий код и посмотрим. Нулевые значения заполним медианой сотни самых частых ненулевых значений (выбрано экспериментальным путём).

Таки оба признака со временем оказались в первой десятке важных для модели. Неплохо.

In [ ]:
reviews = data['Reviews'].apply(
    lambda x: x[2:-2].split('], [')[1][1:-1].split("', '") if pd.isna(x) == False else [0])

# заполним константой, в расчётах она всё равно превратится в медиану.
reviews = reviews.apply(lambda x: x if x[0] != '' else ['01/01/1900'])

deltanow = []

for i in reviews:
    if len(i) > 1:
        i[0] = datetime.strptime(i[0], '%m/%d/%Y')
        i[1] = datetime.strptime(i[1], '%m/%d/%Y')
        d = abs((i[1]-datetime.today())).days
        deltanow.append(d)
    else:
        deltanow.append(0)

data['NowDelta'] = deltanow
data['NowDelta'] = data['NowDelta'].apply(lambda x: pd.Series(
    data['NowDelta'].value_counts()[1:100].index).median() if x == 0 else x)

In [ ]:
data.head()

# EDA 


### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10, 7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] == 'London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)
sns.heatmap(df_train.drop(['sample'], axis=1).corr(), annot=True)

 Посмотрим на разницу между двумя последними отзывами. 
Как-то очень много выбросов. Значимой корреляции с Ranking нет. Интересно, чем они могут быть обусловлены. Может, это первый и последний отзывы, а не два последних? В общем, надо разбираться, в нашу задачу сейчас это не входит.

In [ ]:
data['ReviewTimeDelta'].hist(bins=20)
data['ReviewTimeDelta'].max()

In [ ]:
sns.boxplot(data['ReviewTimeDelta'])

Это время с последнего отзыва. 

In [ ]:
data['NowDelta'].hist(bins=20)

И время между отзывами.

In [ ]:
data['ReviewTimeDelta'].hist(bins=20)

Ну, этот график просто посмотреть разные соотношения, поиграться.

In [ ]:
plt.figure(figsize=(5, 5))
sns.jointplot(y='ReviewTimeDelta', x='Ranking',
              data=data, kind='reg')

#### Удаляем нецифровые данные и заполняем нулями то, что осталось незаполненным

In [ ]:
for column in data:
    if data[column].dtypes != 'float64' and data[column].dtypes != 'int64' and data[column].dtypes != 'uint8':
        data = data.drop([column], axis=1)
data = data.fillna(0)

Попробуем провести нормализацию.

Как оказалось, эффект на MAE это произвело незначительный, однако эти данные вошли в 20 важнейших для модели.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
datanorm = pd.DataFrame(scaler.fit_transform(data))
datanorm.columns = data.columns

# Удалим все dummy-переменные, а также целевую переменную и sample
for i in datanorm:
    if i not in ['Ranking', 'Number of Reviews', 'Price Range', 'Cuisine_assort', 'ReviewTimeDelta', 'NowDelta']:
        datanorm = datanorm.drop([i], axis=1)

new_col_name = []  # переименуем колонки, чтобы добавить их к основному датафрейму
for i in datanorm:
    i = 'norm_'+i
    new_col_name.append(i)
datanorm.columns = new_col_name


# Добавим колонки с нормализованными данными к основному датафрейму
for i in datanorm:
    data[i] = datanorm[i]

data.head()

In [ ]:
df_preproc = data

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics  # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(
    n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10, 10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')

Как видим, большая часть новых признаков входит в 20 самых важных. Dummy-признаки городов туда входят, а вот кухонь - нет, так что, думаю, их можно удалить из датафрейма без особого вреда.

### Посмотрим на корреляцию самых важных признаков


In [ ]:
df1 = data[(feat_importances.nlargest(20).index)]
df1.head()
plt.rcParams['figure.figsize'] = (15, 10)
sns.heatmap(df1.corr(), annot=True, cmap='coolwarm')

Высокая корреляция между ReviewTimeDelta и NowDelta, что довольно логично. Можно бы один из этих признаков убрать, но точность модели тогда снижается. Так что пусть будут оба.

Умеренная положительная корреляция между количеством отзывов и ассортиментом кухонь в ресторане.
И некоторая отрицательная корреляция между ассортиментом кухонь и Ranking (то есть, чем больше кухонь, тем популярнее ресторан). Интересно. Наверное, не могут выбрать и уходят голодными.

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission.head(10)

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
test_data['Raiting'] = predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission


sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)